---

## Authentication System Database Schema

This database schema is designed to handle user authentication, including features such as email/password signup, social login options, email validation, and password recovery. The schema includes three main tables: `Users`, `Social Logins`, and `Password Resets`.

### 1. **Users Table**

**Purpose**:  
Stores the core credentials and profile information of each user. This is the central entity in the authentication system.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `user_id`                 | INTEGER           | Primary key, auto-incremented unique identifier for each user. |
| `email`                   | VARCHAR(255)      | User's email address, must be unique.                         |
| `password_hash`           | VARCHAR(255)      | Hash of the user's password.                                  |
| `profile_type`            | ENUM('admin', 'user', 'guest') | Type of user profile, with default as 'user'.          |
| `email_verified`          | BOOLEAN           | Boolean flag to indicate if the email address has been verified. |
| `email_verification_token`| VARCHAR(255)      | Token used for the email verification process.                 |
| `created_at`              | DATETIME          | Timestamp when the user account was created.                   |
| `updated_at`              | DATETIME          | Timestamp when the user account was last updated.              |

### 2. **Social Logins Table**

**Purpose**:  
Handles user authentication through social media platforms, linking social media accounts to user profiles in the `Users` table.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `social_id`               | INTEGER           | Primary key, auto-incremented unique identifier for each social login record. |
| `user_id`                 | INTEGER           | Foreign key linking back to the `Users` table.               |
| `provider`                | VARCHAR(50)       | Name of the social media platform (e.g., 'Google', 'Facebook'). |
| `social_user_id`          | VARCHAR(255)      | Unique identifier provided by the social media platform.     |
| `created_at`              | DATETIME          | Timestamp when the social login was created.                  |
| `updated_at`              | DATETIME          | Timestamp when the social login was last updated.             |

### 3. **Password Resets Table**

**Purpose**:  
Manages the process for users to reset their passwords, including storing reset tokens and their expiration times.

**Fields**:

| Field Name                | Data Type         | Description                                                  |
|---------------------------|-------------------|--------------------------------------------------------------|
| `reset_id`                | INTEGER           | Primary key, auto-incremented unique identifier for each password reset record. |
| `user_id`                 | INTEGER           | Foreign key linking back to the `Users` table.               |
| `reset_token`             | VARCHAR(255)      | Token used to authenticate the password reset request.       |
| `expiration_date`         | DATETIME          | Expiry date and time for the reset token.                    |
| `created_at`              | DATETIME          | Timestamp when the reset request was created.                 |
| `used`                    | BOOLEAN           | Boolean flag to indicate whether the reset token has been used. |

---


In [12]:
from sqlalchemy import create_engine, text

In [13]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [16]:
 tables = ['users','social_logins','password_resets']

with engine.connect() as connection:
    for table in tables:
        engine.execute(f'DROP TABLE IF EXISTS {table}')
    

2024-04-16 07:13:08,287 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS users
2024-04-16 07:13:08,290 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 07:13:08,292 INFO sqlalchemy.engine.Engine COMMIT
2024-04-16 07:13:08,295 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS social_logins
2024-04-16 07:13:08,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 07:13:08,300 INFO sqlalchemy.engine.Engine COMMIT
2024-04-16 07:13:08,303 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS password_resets
2024-04-16 07:13:08,304 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-16 07:13:08,307 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
# Define raw SQL queries for table creation using text()
create_users_table = text("""
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    profile_type DEFAULT 'creator',
    email_verified BOOLEAN NOT NULL DEFAULT 0,
    email_verification_token VARCHAR(255),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
""")

create_social_logins_table = text("""
CREATE TABLE IF NOT EXISTS social_logins (
    social_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    provider VARCHAR(50) NOT NULL,
    social_user_id VARCHAR(255) NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
""")

create_password_resets_table = text("""
CREATE TABLE IF NOT EXISTS password_resets (
    reset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    reset_token VARCHAR(255) NOT NULL,
    expiration_date DATETIME NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    used BOOLEAN NOT NULL DEFAULT 0
);
""")

In [18]:
# Execute SQL statements
with engine.connect() as connection:
    connection.execute(create_users_table)
    connection.execute(create_social_logins_table)
    connection.execute(create_password_resets_table)

2024-04-16 07:13:15,761 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    profile_type DEFAULT 'creator',
    email_verified BOOLEAN NOT NULL DEFAULT 0,
    email_verification_token VARCHAR(255),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

2024-04-16 07:13:15,765 INFO sqlalchemy.engine.Engine [generated in 0.00413s] ()
2024-04-16 07:13:15,771 INFO sqlalchemy.engine.Engine COMMIT
2024-04-16 07:13:15,775 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS social_logins (
    social_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    provider VARCHAR(50) NOT NULL,
    social_user_id VARCHAR(255) NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)

In [19]:
# Query to list all tables in a SQLite database
list_tables_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
with engine.connect() as connection:
    result = connection.execute(list_tables_query)
    tables = result.fetchall()
    for table in tables:
        print(table[0])

2024-04-16 07:13:26,081 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table';
2024-04-16 07:13:26,084 INFO sqlalchemy.engine.Engine [raw sql] ()
sqlite_sequence
users
social_logins
password_resets


## Adding a New User

In [11]:
insert_query = text('''INSERT INTO 
                    users (email, 
                           password_hash, 
                           profile_type, 
                           email_verified,
                           email_verification_token)

                    VALUES ('user@example.com',
                            'hashed_password_here',
                            'user', 
                            0, 
                            'verification_token_here');''')
# Execute the query
with engine.connect() as connection:
    result = connection.execute(insert_query)

2024-04-15 23:24:44,974 INFO sqlalchemy.engine.Engine INSERT INTO 
                    users (email, 
                           password_hash, 
                           profile_type, 
                           email_verified,
                           email_verification_token)

                    VALUES ('user@example.com',
                            'hashed_password_here',
                            'user', 
                            0, 
                            'verification_token_here');
2024-04-15 23:24:44,975 INFO sqlalchemy.engine.Engine [cached since 14.92s ago] ()
2024-04-15 23:24:44,978 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO 
                    users (email, 
                           password_hash, 
                           profile_type, 
                           email_verified,
                           email_verification_token)

                    VALUES ('user@example.com',
                            'hashed_password_here',
                            'user', 
                            0, 
                            'verification_token_here');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## Verifying user email
SET email_verified = 1, email_verification_token = NULL
WHERE user_id = 1; 


-- Assume user_id = 1 is the user to verify

In [20]:
import datetime
import hashlib

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

In [ ]:
def register_user(email,password):
    hashed_password = hash_password(password)
    register_query = '''INSERT INTO users (email, 
                           password_hash, 
                           profile_type, 
                           email_verified,
                           email_verification_token)